##Importing The Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
tf.__version__

'2.5.0'

##Importing The Dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
Y = dataset.iloc[:,-1].values


In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(Y)

[1 0 1 ... 1 1 0]


##Encoding Categorical Data

###Label Encdoing The Gender

In [6]:
from sklearn.preprocessing import LabelEncoder
LabelEncoder = LabelEncoder()
X[:,2] = LabelEncoder.fit_transform(X[:,2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


###One Hot Encoding The 'Geography' Column

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


##Splitting Dataset

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 0)

##Feature Scaling

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Building The ANN

###Initializing The ANN

In [12]:
ann = tf.keras.models.Sequential()

###Adding The Input Layer and The First Hidden Layer

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

###Adding The Second Hidden Layer

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

###Adding The Output Layer

In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

##Training The ANN

###Compiling The ANN

In [16]:
ann.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])

###Trining The ANN on The Training Set

In [17]:
ann.fit(X_train, Y_train, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5309 - accuracy: 0.7569
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4620 - accuracy: 0.8001
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4431 - accuracy: 0.8081
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4335 - accuracy: 0.8135
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4277 - accuracy: 0.8159
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4237 - accuracy: 0.8191
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4204 - accuracy: 0.8206
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4182 - accuracy: 0.8242
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4156 - accuracy: 0.8253
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4137 - accura

##Making The Predictions and Evaluating The Model

###Predicting The Results of a Single Observation

In [18]:
ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))

array([[0.01912645]], dtype=float32)

###Predicting The Test Set

In [19]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), Y_test.reshape(len(Y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


###Creating The Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, y_pred)
print("The Confusion Matrix:\n", cm)

The Confusion Matrix:
 [[1527   68]
 [ 208  197]]


###The Accuracy Score

In [21]:
print("The Accuracy Score:\n",accuracy_score(Y_test, y_pred))

The Accuracy Score:
 0.862
